In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
# Adam is faster than SGD (Stochastic Gradient Descent)
from torch.optim import Adam

import lightning as L
from torch.utils.data import TensorDataset, DataLoader

c:\python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
class LSTMbyHand(L.LightningModule):
    """
    Class created in order to initialize a LSTM RNN model that predicts a stock's value in the 5th day,
    having other 4 days as inputs
    """
    
    def __init__(self):
        """
        Create and initialize Weight and Bias tensors
        """
        # First, call the initialization method for the parent clas, LightningModule
        super().__init__()
        
        # We initialize the weights and biases around 0 (we make a normal distribution around 0 and a standard 
        # deviation around 1)
        mean = torch.tensor(0.0)
        std = torch.tensor(1.0)
        
        # We initialize the weights/biases for all the gates (all the cells in the LSTM unit)
        self.wlr1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wlr2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.blr1 = nn.Parameter(torch.tensor(0., requires_grad=True))
                                 
        self.wpr1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wpr2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bpr1 = nn.Parameter(torch.tensor(0., requires_grad=True))
                                 
        self.wp1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wp2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bp1 = nn.Parameter(torch.tensor(0., requires_grad=True))
                                
        self.wo1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wo2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bo1 = nn.Parameter(torch.tensor(0., requires_grad=True))
    
    def lstm_unit(self, input_value, long_memory, short_memory):
        """
        Do the LSTM math
        """
        # Calculates the percentage of the long-term memory to remember
        long_remember_percent = torch.sigmoid((short_memory * self.wlr1) + (input_value * self.wlr2) + self.blr1)
        
        # Creates a new, potential long-term memory and determines what percentage of it to remember
        potential_remember_percent = torch.sigmoid((short_memory * self.wpr1) + (input_value * self.wpr2) + self.bpr1)
        potential_memory = torch.tanh((short_memory * self.wp1) + (input_value * self.wp2) + self.bp1)
        
        # Update the long-term memory
        updated_long_memory = ((long_memory * long_remember_percent) + (potential_remember_percent * potential_memory))
        
        # Create a new short-term memory and determine what percentage to remember
        output_percent = torch.sigmoid((short_memory * self.wo1) + (input_value * self.wo2) + self.bo1)
        updated_short_memory = torch.tanh(updated_long_memory) * output_percent
        
        # Return the long and short-term memories
        return([updated_long_memory, updated_short_memory])
    
    def forward(self, input):
        """
        Make a forward pass through unrolled LSTM
        """
        long_memory = 0
        short_memory = 0
        day1 = input[0]
        day2 = input[1]
        day3 = input[2]
        day4 = input[3]
        
        # Call LSTM with initial values (Day 1)
        long_memory, short_memory = self.lstm_unit(day1, long_memory, short_memory)
        
        # Call LSTM with values from Day 2
        long_memory, short_memory = self.lstm_unit(day2, long_memory, short_memory)
        
        # LSTM with values from Day 3
        long_memory, short_memory = self.lstm_unit(day3, long_memory, short_memory)
        
        # LSTM with values from Day 4
        long_memory, short_memory = self.lstm_unit(day4, long_memory, short_memory)
        
        return short_memory
    
    def configure_optimizers(self):
        """
        Configure Adam optimizer
        """
        return Adam(self.parameters())
    
    def training_step(self, batch, batch_idx):
        """
        Calculate loss and log training progress
        """
        input_i, label_i = batch
        output_i = self.forward(input_i[0])
        loss = (output_i - label_i)**2
        
        self.log("train_loss", loss)
        
        if (label_i == 0):
            self.log("out_0", output_i)
        else:
            self.log("out_1", output_i)
            
        return loss

In [42]:
model = LSTMbyHand()
print("\nNow let's compare the observed and predicted values...")
print("Company A: Observed = 0, Predicted = ", model(torch.tensor([0., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values...
Company A: Observed = 0, Predicted =  tensor(0.1096)


In [43]:
print("Company B: Observed = 1, Predicted = ", model(torch.tensor([1., 0.5, 0.25, 1.])).detach())

Company B: Observed = 1, Predicted =  tensor(0.1242)


In [46]:
# Input data, days 1..4
inputs = torch.tensor([[0., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.]])
# Label data, what we want to predict (0 for company A, 1 for company B)
labels = torch.tensor([0., 1.])